In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [26]:
df = pd.read_csv("train.csv")
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [27]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace=True)

In [28]:
df.head(2)

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C

In [29]:
X = df.drop(['Survived'], axis = 1)
X.head(1)

Pclass   Sex   Age  SibSp  Parch  Fare Embarked
0       3  male  22.0      1      0  7.25        S

In [30]:
y = df['Survived']

In [31]:
y.head(1)

0    0
Name: Survived, dtype: int64

In [32]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=10,stratify=y)

In [33]:
X_train.shape

(712, 7)

In [34]:
df.shape

(891, 8)

In [35]:
X_test.shape

(179, 7)

In [36]:
X_train.head(1)

Pclass     Sex  Age  SibSp  Parch  Fare Embarked
415       3  female  NaN      0      0  8.05        S

In [44]:
X_train.sample(2)

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
479       3  female   2.0      0      1  12.2875        S
456       1    male  65.0      0      0  26.5500        S

In [46]:
len(df.index)

891

In [48]:
df.shape

(891, 8)

In [59]:
df.head(0)

Empty DataFrame
Columns: [Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked]
Index: []

In [60]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [57]:
X_train.head(0)

Empty DataFrame
Columns: [Pclass, Sex, Age, SibSp, Parch, Fare, Embarked]
Index: []

In [58]:
X_train.head(1)

Pclass     Sex  Age  SibSp  Parch  Fare Embarked
415       3  female  NaN      0      0  8.05        S

### Applying the first part of Pipeline - Imputing the missing values 

In [39]:
## imputing means assigning

In [40]:
# Creating the imputer transformer - 

tf1 = ColumnTransformer([
    ('tf1_age' , SimpleImputer(), [2]),
    ('tf1_embarked' , SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

In [55]:
# creating one hot encoding transformer - 

tf2 = ColumnTransformer([
    ('tf2_sex', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1, 6] )
], remainder='passthrough')

* here, we have given the indexing and not the names of the columns becuase further it will create the problems so follow the indexing method

In [61]:
# creating the Scaling transformer - 

tf3 = ColumnTransformer([
    ('tf3_scale' , MinMaxScaler(), slice(0,10))
])

* here we are using MinMax scaler because further we will be applying 'Feature Selection', if Feature Selection was not to be applied then we could have used Standard Scaler but if Feature Selection is to be performed then MinMax Scaler is to be used and not Standard Scaler
* slice(0,8) means we have taken all the columns and have applied scaling on all the columns
* 10 columns because after One Hot Encoding the columns will be 10 in number.

In [62]:
# Feature Selection - 
tf4 = SelectKBest(score_func=chi2 , k=8)

In [63]:
# train the model - 
tf5 = DecisionTreeClassifier()

### Creating the Pipeline - 

In [64]:
pipe = Pipeline([
    ('tf1', tf1),
    ('tf2', tf2),
    ('tf3', tf3),
    ('tf4', tf4),
    ('tf5', tf5)
])

In [65]:
# alternate syntax - creating the Pipeline through 'make Pipeline' - 
pinem = make_pipeline(tf1,tf2,tf3,tf4,tf5)

In [70]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tf1_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('tf1_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Embarked'])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tf2_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tf3',
                 ColumnTransformer(transformers=[('tf3_scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
 

In [73]:
from sklearn import set_config
set_config(display='diagram')

## write this code to get the below diagram

In [72]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tf1_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('tf1_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Embarked'])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tf2_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tf3',
                 ColumnTransformer(transformers=[('tf3_scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000144ED2F2040>)),
                ('tf5', DecisionTreeClassifier())])

In [74]:
# Predict - 
y_pred = pipe.predict(X_test)

In [75]:
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0], dtype=int64)

In [77]:
pipe.named_steps

{'tf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('tf1_age', SimpleImputer(), ['Age']),
                                 ('tf1_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  ['Embarked'])]),
 'tf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('tf2_sex',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tf3': ColumnTransformer(transformers=[('tf3_scale', MinMaxScaler(),
                                  slice(0, 10, None))]),
 'tf4': SelectKBest(k=8, score_func=<function chi2 at 0x00000144ED2F2040>),
 'tf5': DecisionTreeClassifier()}

In [78]:
pipe.named_steps['tf1'].transformers_

[('tf1_age', SimpleImputer(), ['Age']),
 ('tf1_embarked', SimpleImputer(strategy='most_frequent'), ['Embarked']),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [79]:
pipe.named_steps['tf2'].transformers_

[('tf2_sex', OneHotEncoder(handle_unknown='ignore', sparse=False), [1, 6]),
 ('remainder', 'passthrough', [0, 2, 3, 4, 5])]

In [86]:
pipe.named_steps['tf1'].transformers_[1][1]

SimpleImputer(strategy='most_frequent')

In [87]:
pipe.named_steps['tf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

* so here the most frequent value is 'S'
* the above code is used to find the results of what we have applied

In [89]:
pipe.named_steps['tf1'].transformers_[0]

('tf1_age', SimpleImputer(), ['Age'])

In [91]:
pipe.named_steps['tf1'].transformers_[0][1].statistics_

array([29.88289655])

In [92]:
from sklearn.metrics import accuracy_score

In [93]:
accuracy_score(y_test, y_pred)

0.6368715083798883

* here the accuracy score is less becuase we have removed some of the columns through Feature Selection. 
* Feature Selection can sometimes not be beneficial for us like here in this case.

In [ ]:
# cross-validation
# grid search left in this tutorial